### This notebook is for M2 (NTS or Summit)
### We get the LUT force info from the EFD
### we compare the M2 simulator LUT implementation with Harris csv files, for one zenith angle.

In [ ]:
from openpyxl import load_workbook

from astropy.time import Time
from datetime import timedelta, datetime
from lsst_efd_client import EfdClient

import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import pandas as pd
import os

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.jet();

In [ ]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [ ]:
if summit:
    start = Time('2021-03-25T15:19:00', scale = 'tai') #these are after we applied the +2/-2N forces.
    end   = Time('2021-03-25T15:21:00', scale = 'tai') 
    client = EfdClient('summit_efd')
else:
    # NCSA
    start = Time('2021-03-23T00:50:00', scale = 'tai') 
    end   = Time('2021-03-23T00:52:00', scale = 'tai') 
    client = EfdClient('ncsa_teststand_efd')

csc_index = 1
aF = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
tF = await client.select_time_series('lsst.sal.MTM2.tangentForce', '*', start, end, csc_index)
temp = await client.select_time_series('lsst.sal.MTM2.temperature', '*', start, end, csc_index)
zA = await client.select_time_series('lsst.sal.MTM2.zenithAngle', '*', start, end, csc_index)

In [ ]:
len(aF)

### do things change with time, for example, LUT gravity?

In [ ]:
fig, ax = plt.subplots(figsize=(15,3))
plt.plot(90-zA.measured)
plt.title('elevation ')
plt.grid() 

In [ ]:
fig, ax = plt.subplots(figsize=(15,3))
for i in range(72):
    plt.plot(aF['lutGravity%d'%i])
plt.title('gravity LUT')
plt.grid() 

In [ ]:
fig, ax = plt.subplots(figsize=(15,3))
plt.plot([temp['ring%d'%i][0] for i in range(12)], label='ring')
plt.plot([temp.intake0[0], temp.intake1[-1]],'-o', label = 'intake')
plt.plot([temp.exhaust0[0], temp.exhaust1[-1]],'-*', label = 'exhaust')
plt.title('temperature')
plt.legend()
plt.grid() 

#### FinalHandingLUTs vs FinalOpticalLUTs 
* https://github.com/lsst-ts/ts_mtm2_cell/tree/master/configuration/lsst-m2/config/parameter_files/luts
* There are 8 files in each subfolder: FinalHandingLUTs vs FinalOpticalLUTs. The only file that is different is F_F.csv
* This is easily understandable, because the optical LUT cares about mirror shape at any zenith angle. While the other one is used for on-cart rotation.
* Which one did we use for the 2020 Feb summit test? Te-Wei says we used HandlingLUTs.
* Do we know how to switch? Yes. By the high level configuration file.

In [ ]:
LUTfolder = '%s/notebooks/M2_summit_2003/harrisLUT/FinalHandlingLUTs'%(os.environ["HOME"])
dfe = pd.read_csv('%s/F_E.csv'%LUTfolder)
FE = np.float64(dfe)
df0 = pd.read_csv('%s/F_0.csv'%LUTfolder)
F0 = np.float64(df0)
dff = pd.read_csv('%s/F_F.csv'%LUTfolder)
FF = np.float64(dff)
dfa = pd.read_csv('%s/F_A.csv'%LUTfolder)
FA = np.float64(dfa)
elev = np.float64(dfe.keys()) # the columns are labelled with elevation angle.
dtr = pd.read_csv('%s/Tr.csv'%LUTfolder, header=None)
Tr = np.float64(dtr)
dtu = pd.read_csv('%s/Tu.csv'%LUTfolder, header=None)
Tu = np.float64(dtu)
dtx = pd.read_csv('%s/Tx.csv'%LUTfolder, header=None)
Tx = np.float64(dtx)
dty = pd.read_csv('%s/Ty.csv'%LUTfolder, header=None)
Ty = np.float64(dty)

LUTfolder = '%s/notebooks/M2_summit_2003/harrisLUT/FinalOpticalLUTs'%(os.environ["HOME"])
dff = pd.read_csv('%s/F_F.csv'%LUTfolder)
FF1 = np.float64(dff)

In [ ]:
#73 columns, each column for one angle (-270 - 90)
print(FE.shape, F0.shape, FF.shape, FA.shape, Tr.shape, Tu.shape, Tx.shape, Ty.shape)
#FA (actuator weight component) is the only one that includes the tangent links. Why???

In [ ]:
# The below refers to the correctability document from Harris -
# according to Harris page 26, 0-90 is telescope operating range.
# according to Harris page 32, 0 deg is horizon, 90 deg is zenith.
# positive z goes into M2, to the sky. The angle refers to the z axis.
# Harris page 29 shows it cares a lot about zenith, which is z pointing up (90 deg from x axis.)

fig, ax = plt.subplots(2,3, figsize=(15,8))
s1 = ax[0][0].plot(elev, F0.transpose())
ax[0][0].set_title('F0: 0g correction')
ax[0][0].grid()
s2 = ax[0][1].plot(elev, FE.transpose())
ax[0][1].set_title('FE: elevation component')
ax[0][1].grid()
s3 = ax[1][0].plot(elev, FF1.transpose())
ax[1][0].set_title('FF1: figure correction')
ax[1][0].grid()
s4 = ax[1][1].plot(elev, FA[:72,:].transpose())
ax[1][1].set_title('FA: actuator weight')
ax[1][1].grid()

ax[0][2].plot(Tr, label='Tr')
ax[0][2].plot(Tu, label='Tu')
ax[0][2].legend()
ax[1][2].plot(Tx, label='Tx')
ax[1][2].plot(Ty, label='Ty')
ax[1][2].legend();

In [ ]:
sum(FA[:72,0])

In [ ]:
np.std(FF1[:,0])

In [ ]:
FA[72:,0]

In [ ]:
M2Weight = 15578 #Newton
def lookUpGravity(summit, lutInAngle, FE, FA,elevA):
    '''
    input:
        lutInAngle is the angle used to define the LUT, which is the elevation angle.
        elevA is the actual measured elevation angle.
    output:
        gravity LUT force
    '''
    myfe = np.zeros(78)
    myfa = np.zeros(78)
    myf0 = np.zeros(78)
    myff = np.zeros(78)
    for i in range(72):
        myfe[i] = np.interp(elevA, lutInAngle, FE[i,:])
        myf0[i] = np.interp(elevA, lutInAngle, F0[i,:])
        myff[i] = np.interp(elevA, lutInAngle, FF[i,:])
    F2 = M2Weight* np.sin(np.radians(90-elevA))/2/np.sqrt(3)
    myfe[73] = F2
    myfe[74] = F2
    myfe[76] = -F2
    myfe[77] = -F2
    for i in range(78):
        myfa[i] = np.interp(elevA, lutInAngle, FA[i,:])
    myf = myfe + myfa 
    if summit: 
        #with TeWei's summit implementation, we also included f0 and ff in lutGravity
        myf = myf + myf0 + myff
    else:
        #The tangent forces in the M2 simulator has a known bug (DM-28515) - all the tangent force should have reversed sign.
        myfe[72:] = -myfe[72:]
    return myf

def lookUpTemperature(Tr, Tx, Ty, Tu, tempInv, efdT):
    '''
    input:
        efdT is the list of temperature values ordered as [0-11, intake0, intake1, exhaust0, exhaust1]
    output:
        thermal LUT force
    '''
    #Order temperature data based on a12_temperature.ipynb
    binT = efdT[[0,1,2,3, 12,15,14,13, 8,9,10,11, 4,5,6,7]]
    lutT = binT[ [1, 2, 3, 12, 9, 8, 13, 14, 15, 11, 10, 0]]
    tref = 21 #deg C. 
    tcoef = tempInv.dot(lutT-tref)
    
    for i in range(72):
        myft = tcoef[0]*Tr + tcoef[1]*Tx + tcoef[2]*Ty + tcoef[3]*Tu
    myf = np.squeeze(myft)
    return myf

In [ ]:
#check lutGravity against expectation
dAngle = -5
myFE1 = [aF['lutGravity%d'%i][0] for i in range(72)] 
myFE2 = [tF['lutGravity%d'%i][0] for i in range(6)]
Felev = lookUpGravity(summit, elev, FE, FA, 90-zA.measured[0])
FelevD = lookUpGravity(summit, elev, FE, FA, 90-zA.measured[0]+dAngle)
fig, ax = plt.subplots(3,1, sharex = False, figsize=(15,6))
ax[0].plot(myFE1 ,'-ro', label='EFD')
ax[0].plot(Felev[:72], label='expected')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial, max diff = %.1fN'%max(abs(myFE1-Felev[:72])))
ax[1].plot(myFE2,'-ro', label='EFD')
ax[1].plot(Felev[72:], label='expected')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent')
plt.suptitle('LUT Gravity');
#on the summit: the tangent mismatch should be due to that the calculation is based on a perfect model

ax[2].plot(Felev[:72] - FelevD[:72], label = 'force change (%.1f --> %.1f) deg'%(90-zA.measured[0], 90-zA.measured[0]+dAngle))
ax[2].grid();
ax[2].legend();

In [ ]:
tempInv = np.loadtxt('%s/notebooks/M2_summit_2003/harrisLUT/temp_inv.txt'%(os.environ["HOME"]))
tempInv.shape

### Next, the question is, how are the 16 temps ordered. Accordiing to Te-Wei
* according to https://github.com/lsst-ts/ts_xml/blob/0708391562e358cd5a11818420f24f9f44a5d6e9/sal_interfaces/MTM2/MTM2_Telemetry.xml#L184, in the EFD, the ordering is Ring temperatures: LG2-1, LG2-2, LG2-3, LG2-4, LG3-1, LG3-2, LG3-3, LG3-4, LG4-1, LG4-2, LG4-3, and LG4-4.
* https://github.com/lsst-ts/ts_mtm2/issues/17 To use them with tempInv matrix, we need to order them as "LG2-2, LG2-3, LG2-4, LG3-1, LG4-2, LG4-1, LG3-2, LG3-3, LG3-4, LG4-4, LG4-3, LG2-1".
* https://jira.lsstcorp.org/browse/DM-23523?filter=17301 an alternative is to order them as (same ordering as in the binary outputs)
               LG2-1
               LG2-2
               LG2-3
               LG2-4
               Intake #1
               Exhaust #1
               Exhaust #2
               Intake #2
                LG4-1
                LG4-2
              LG4-3
              LG4-4
              LG3-1
              LG3-2
              LG3-3
              LG3-4
then use the index array: [1, 2, 3, 12, 9, 8, 13, 14, 15, 11, 10, 0]. (index begins from 0). 

In [ ]:
#The intake and exhaust temps are not used for LUT
tt = [temp['ring%d'%i][0] for i in range(4)] + [0,0,0,0] + \
    [temp['ring%d'%i][0] for i in range(8,12)] + [temp['ring%d'%i][0] for i in range(4,8)]
idx = [1, 2, 3, 12, 9, 8, 13, 14, 15, 11, 10, 0]
myt= np.array([tt[i] for i in idx])

In [ ]:
tref = 21 #deg C. 
tcoef = tempInv.dot(myt-tref)
FT = tcoef[0]*Tr + tcoef[1]*Tx + tcoef[2]*Ty + tcoef[3]*Tu

In [ ]:
#check lutTemperature against expectation
myFT1 = [aF['lutTemperature%d'%i][0] for i in range(72)] 
myFT2 = [tF['lutTemperature%d'%i][0] for i in range(6)]
fig, ax = plt.subplots(2,1, sharex = False, figsize=(15,6))
ax[0].plot(myFT1 ,'-ro', label='EFD')
ax[0].plot(FT[:,0], label='expected')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial, max diff = %.1fN'%max(abs(FT[:,0]-myFT1)))
ax[1].plot(myFT2,'-ro', label='EFD')
ax[1].plot([0]*6, label='expected')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent, max diff = %.1fN'%max(abs(np.array(myFT2))))
plt.suptitle('LUT Temperature');

In [ ]:
aa = np.loadtxt('%s/notebooks/M2_FEA/data/M2_1um_72_force.txt'%os.environ['HOME'])
# these are now in M2 CS (used to be in M2 FEA CS)
xact = aa[:,1]
yact = aa[:,2]

In [ ]:
#plot the balance forces
myFB1 = np.array([aF['hardpointCorrection%d'%i][0] for i in range(72)] )
myFB2 = np.array([tF['hardpointCorrection%d'%i][0] for i in range(6)])
fig, ax = plt.subplots(2,1, sharex = False, figsize=(15,6))
ax[0].plot(myFB1 ,'-ro', label='EFD')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial')
ax[1].plot(myFB2,'-ro', label='EFD')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent')
plt.suptitle('FB forces');

In [ ]:
#plot the applied forces
myFA1 = [aF['applied%d'%i][0] for i in range(72)] 
myFA2 = [tF['applied%d'%i][0] for i in range(6)]
fig, ax = plt.subplots(2,1, sharex = False, figsize=(15,6))
ax[0].plot(myFA1 ,'-ro', label='EFD')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial')
ax[1].plot(myFA2,'-ro', label='EFD')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent')
plt.suptitle('Applied forces');

In [ ]:
#check the sum of the different categories of forces against the measured forces
myFM1 = [aF['measured%d'%i][0] for i in range(72)] 
myFM2 = [tF['measured%d'%i][0] for i in range(6)]
fig, ax = plt.subplots(2,1, sharex = False, figsize=(15,6))

sumF1 = np.array(myFE1)+myFT1+myFB1+myFA1 
ax[0].plot(myFM1 ,'-ro', label='measured')
ax[0].plot(sumF1, label='sum')
ax[0].grid()
ax[0].legend()
ax[0].set_title('Axial, max diff = %.1fN'%max(abs(sumF1-myFM1)))

sumF2 = np.array(myFE2)+myFT2+myFB2+myFA2
ax[1].plot(myFM2,'-ro', label='measured')
ax[1].plot(sumF2, label='sum')
ax[1].grid()
#ax[1].set_ylim([-1,1])
ax[1].legend()
ax[1].set_title('Tangent, max diff = %.1fN'%max(abs(sumF2-myFM2)))
plt.suptitle('Total forces');

In [ ]:
#with the simulator, this is expected to be a normal distribution with rms = 0.5N
#for real M2, axial force accuracy = 1N, tangent force accuracy = 10N
plt.hist(np.hstack((sumF1-myFM1, sumF2-myFM2)),10);